# Getting Started
## Prerequisites
An understanding of the following packages is required to better understand the workings of SageRender.
* [Phiera](https://github.com/nike-inc/phiera)
* [SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/)

## Step 1: Phiera Configuration
Create a `hiera.yaml` with the following sections:
* backends
  Define the supported backends for the existing hierarchical configuration. Phiera supports, yaml, yaml.enc (YAML files encrypted by SOPS), and json backends.
* context
  Define context variables that used to define the hierarchy at runtime. These can be looked up in the configuration files using `%{variable_name}` syntax where variable_name is the context variable.
* hierarchy
  Define the hierarchy to adhere to in the order of precedence. *The values defined in the current path overrides the values of all the paths defined below it.*
* datadir definition for each backend
  Define the source of the files for each backend. This is defined using the following structure:
  ```yaml
  yaml: # Name of the backend
    datadir: path/to/config/folder  # path to the configuration folder from pwd of execution environment
  ```

![Simple Hierarchy](../extras/simple-hierarchy.png)
Let's consider a simple example using the above hierarchy defined in [hiera.yaml](../examples/hiera.yaml).


## Step 2: Configure a simple single step pipeline

The pipeline is configured in [common.yaml](../examples/config/common.yaml) between lines 15-30. The environment specific configuration is then configured in the environment-specific YAML files.

In addition to the context variables defined in hiera.yaml, we can pass in additional context variables which can then be referenced in the YAML file using `%{variable_name}` syntax. In the example, `team` is configured as an additional context variable and this is passed in as part of the CLI arguments.

Each step of the pipeline is configured in two parts:
* Processor<br/>
Defines the execution environment used for the step. Follows the method signature defined in SageMaker Python SDK.

* Step<br/>
Defines the I/O and script used for executing the step. Follows the method signature defined in SageMaker Python SDK.

## Step 3: Upsert Pipeline

### Change pwd to examples

In [1]:
import os

os.chdir("../examples")

### Required only if hiera.yaml is defined elsewhere
If your hiera.yaml is not defined in the same location as the execution enviroment, then set `SAGERENDER_HIERA_FILE`
to point to the path of hiera.yaml file.

To set env var,
`%env SAGERENDER_HIERA_FILE path/to/hiera.yaml`

### Run Upsert Pipeline
In this example, `team` is the additional context variable used. Please set it to your specific team name when running this example notebook.

In [ ]:
!sagerender upsert-pipeline --env test --pipeline-name local-pipeline --team <team-name>

#### Dry Run Mode
Use dry-run to determine the generated config is inline with expected pipeline definition and that there are no syntax errors. When `--dry-run` flag is set, pipeline definition is not upserted.

In [ ]:
!sagerender upsert-pipeline --env test --pipeline-name local-pipeline --team <team-name> --dry-run

#### Local Mode
There is support for local mode for processors that can be run in local. Simply set `--local` flag to run in local mode. This does not upsert a pipeline to SageMaker and uses local execution environment to run the pipeline. Information about SageMaker pipelines local mode can be found [here](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-local-mode.html).

In [ ]:
!sagerender upsert-pipeline --env test --pipeline-name local-pipeline --team <team-name> --local

## Step 4: Run Pipeline
There are a few different ways to run a pipeline on SageMaker.
* Studio UI<br/>
Search the pipeline in the Studio UI by going to the pipeline explorer on the left. Select the pipeline -> Click on Create Execution -> Run the execution.
* AWS CLI<br/>
Using the following command from local or Studio notebook, `aws sagemaker start-pipeline-execution --pipeline-name <pipeline-name> --region <region-name>`
* SageRender CLI<br/>
Using the following command from local or Studio notebook, `sagerender run-pipeline --pipeline-name <pipeline-name>`